In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.metrics import roc_auc_score
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, zscore
from statsmodels.stats.weightstats import ztest
from scipy.stats import chi2_contingency

In [58]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [59]:
df_test = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')
df_train = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')

In [60]:
print('----------------')
print(df_test.head())
print('\n')
print('----------------')
print(df_train)

----------------
       id  CustomerId    Surname  CreditScore Geography  Gender   Age  Tenure  \
0  165034    15773898   Lucchese          586    France  Female  23.0       2   
1  165035    15782418       Nott          683    France  Female  46.0       2   
2  165036    15807120         K?          656    France  Female  34.0       7   
3  165037    15808905  O'Donnell          681    France    Male  36.0       8   
4  165038    15607314    Higgins          752   Germany    Male  38.0      10   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  
0       0.00              2        0.0             1.0        160976.75  
1       0.00              1        1.0             0.0         72549.27  
2       0.00              2        1.0             0.0        138882.09  
3       0.00              1        1.0             0.0        113931.57  
4  121263.62              1        1.0             0.0        139431.00  


----------------
            id  CustomerId       

In [61]:
import numpy as np
import pandas as pd

def add_nulls(df, fraction=0.1):
    df = df.copy()
    num_cells = df.shape[0] * df.shape[1]
    num_nulls = int(num_cells * fraction)
    rows = np.random.randint(0, df.shape[0], num_nulls)
    cols = np.random.randint(0, df.shape[1], num_nulls)

    for row, col in zip(rows, cols):
        df.iat[row, col] = np.nan

    return df

def add_duplicates(df, fraction=0.1):
    df = df.copy()
    num_rows = int(len(df) * fraction)
    duplicate_rows = df.sample(num_rows, replace=True)
    return pd.concat([df, duplicate_rows], ignore_index=True)



In [62]:
df_test = add_nulls(df_test)
df_test = add_duplicates(df_test)

df_train = add_nulls(df_train)
df_train = add_duplicates(df_train)

# Save the modified DataFrames to CSV files
df_train.to_csv("train_data_modified.csv", index=False)
df_test.to_csv("test_data_modified.csv", index=False)

print("Datasets with nulls and duplicates have been saved successfully.")

Datasets with nulls and duplicates have been saved successfully.


In [63]:
train_null = df_train.isnull().sum().sum()

test_null = df_test.isnull().sum().sum()

print(f'Null Count in Train: {train_null}')
print(f'Null Count in Test: {test_null}')

Null Count in Train: 241870
Null Count in Test: 149826


In [64]:

train_duplicates = df_train.duplicated().sum()
test_duplicates = df_test.duplicated().sum()
print(f"Number of duplicate rows in train_data: {train_duplicates}")
print(f"Number of duplicate rows in test_data: {test_duplicates}")

Number of duplicate rows in train_data: 16503
Number of duplicate rows in test_data: 11002


In [66]:
train_null = df_train.isnull().sum().sum()

test_null = df_test.isnull().sum().sum()

print(f'Null Count in Train: {train_null}')
print(f'Null Count in Test: {test_null}')


Null Count in Train: 241870
Null Count in Test: 149826


In [67]:
train_duplicates = df_train.duplicated().sum()
test_duplicates = df_test.duplicated().sum()

print(f"Number of duplicate rows in train_data: {train_duplicates}")
print(f"Number of duplicate rows in test_data: {test_duplicates}")


Number of duplicate rows in train_data: 16503
Number of duplicate rows in test_data: 11002


In [68]:
num_col = df_train.select_dtypes(include=['number']).columns
print(num_col)

Index(['id', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')


In [72]:
from statsmodels.stats.weightstats import ztest

# Extract credit score data for each gender and drop NaNs
male_credit_score = df_train[df_train['Gender'] == 'Male']['CreditScore'].dropna()
female_credit_score = df_train[df_train['Gender'] == 'Female']['CreditScore'].dropna()

# Check for empty groups or zero variance
if len(male_credit_score) == 0 or len(female_credit_score) == 0:
    print("One of the groups has no data.")
elif male_credit_score.std() == 0 or female_credit_score.std() == 0:
    print("One of the groups has zero variance.")
else:
    # Perform Z-test
    z_stat, p_value = ztest(male_credit_score, female_credit_score)

    print(f"Z-test Statistic: {z_stat}")
    print(f"P-value: {p_value}")

    # Interpretation
    if p_value < 0.05:
        print("Reject the null hypothesis: There is a significant difference in CreditScore between males and females.")
    else:
        print("Fail to reject the null hypothesis: No significant difference in CreditScore between males and females.")


Z-test Statistic: 0.346324847272583
P-value: 0.7290985972603619
Fail to reject the null hypothesis: No significant difference in CreditScore between males and females.


In [73]:
from scipy.stats import ttest_ind

# Extract balances for each country and remove NaNs
france_balance = df_train[df_train['Geography'] == 'France']['Balance'].dropna()
germany_balance = df_train[df_train['Geography'] == 'Germany']['Balance'].dropna()

# Check for empty groups or zero variance
if len(france_balance) == 0 or len(germany_balance) == 0:
    print("One of the groups has no data.")
elif france_balance.std() == 0 or germany_balance.std() == 0:
    print("One of the groups has zero variance.")
else:
    # Perform T-test
    t_stat, p_value = ttest_ind(france_balance, germany_balance, equal_var=False)

    print(f"T-test Statistic: {t_stat}")
    print(f"P-value: {p_value}")

    # Interpretation
    if p_value < 0.05:
        print("Reject the null hypothesis: There is a significant difference in Balance between France and Germany.")
    else:
        print("Fail to reject the null hypothesis: No significant difference in Balance between France and Germany.")


T-test Statistic: -351.4492896889327
P-value: 0.0
Reject the null hypothesis: There is a significant difference in Balance between France and Germany.


In [74]:
from scipy.stats import ttest_ind

# Extract balances for each country and remove NaNs
france_balance = df_train[df_train['Geography'] == 'France']['Balance'].dropna()
spain_balance = df_train[df_train['Geography'] == 'Spain']['Balance'].dropna()

# Check for empty groups or zero variance
if len(france_balance) == 0 or len(spain_balance) == 0:
    print("One of the groups has no data.")
elif france_balance.std() == 0 or spain_balance.std() == 0:
    print("One of the groups has zero variance.")
else:
    # Perform T-test
    t_stat, p_value = ttest_ind(france_balance, spain_balance, equal_var=False)

    print(f"T-test Statistic: {t_stat}")
    print(f"P-value: {p_value}")

    # Interpretation
    if p_value < 0.05:
        print("Reject the null hypothesis: There is a significant difference in Balance between France and Spain.")
    else:
        print("Fail to reject the null hypothesis: No significant difference in Balance between France and Spain.")


T-test Statistic: -7.385894520291346
P-value: 1.5346201202565083e-13
Reject the null hypothesis: There is a significant difference in Balance between France and Spain.


In [75]:
from scipy.stats import ttest_ind

# Extract balances for each country and remove NaNs
germany_balance = df_train[df_train['Geography'] == 'Germany']['Balance'].dropna()
spain_balance = df_train[df_train['Geography'] == 'Spain']['Balance'].dropna()

# Check for empty groups, zero variance, and data types
if len(germany_balance) == 0 or len(spain_balance) == 0:
    print("One of the groups has no data.")
elif germany_balance.std() == 0 or spain_balance.std() == 0:
    print("One of the groups has zero variance.")
elif not pd.api.types.is_numeric_dtype(germany_balance) or not pd.api.types.is_numeric_dtype(spain_balance):
    print("One of the groups contains non-numeric data.")
else:
    # Perform T-test
    t_stat, p_value = ttest_ind(germany_balance, spain_balance, equal_var=False)

    print(f"T-test Statistic: {t_stat}")
    print(f"P-value: {p_value}")

    # Interpretation
    if p_value < 0.05:
        print("Reject the null hypothesis: There is a significant difference in Balance between Germany and Spain.")
    else:
        print("Fail to reject the null hypothesis: No significant difference in Balance between Germany and Spain.")


T-test Statistic: 229.05496344878645
P-value: 0.0
Reject the null hypothesis: There is a significant difference in Balance between Germany and Spain.


In [76]:
contingency_table = pd.crosstab(df_train['Gender'], df_train['Geography'])
print(contingency_table)

Geography  France  Germany  Spain
Gender                           
Female      36076    14749  13917
Male        48665    16378  18562


In [77]:
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-squared Statistic: {chi2_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(expected)

if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant association between Gender and Geography.")
else:
    print("Fail to reject the null hypothesis: No significant association between Gender and Geography.")

Chi-squared Statistic: 224.8817450132571
P-value: 1.4707861445919838e-49
Degrees of Freedom: 2
Expected Frequencies:
[[36982.897005   13584.52974445 14174.57325055]
 [47758.102995   17542.47025555 18304.42674945]]
Reject the null hypothesis: There is a significant association between Gender and Geography.
